<a href="https://colab.research.google.com/github/UEPP40/PUM/blob/golon/similar_movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

# Wczytanie danych o ocenach i filmach
ratings = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv')
movies = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv')

# Liczba wszystkich ocen
total_ratings = ratings.shape[0]

# Liczba filmów w bazie
total_movies = movies.shape[0]

# Liczba użytkowników
total_users = ratings['userId'].nunique()

# Średnia liczba ocen na użytkownika
avg_ratings_per_user = total_ratings / total_users

# Średnia liczba ocen na film
avg_ratings_per_movie = total_ratings / total_movies

# Rozkład ocen
ratings_distribution = ratings['rating'].value_counts().sort_index()

# Wyświetlenie wyników
print("Liczba wszystkich ocen:", total_ratings)
print("Liczba filmów w bazie:", total_movies)
print("Liczba użytkowników:", total_users)
print("Średnia liczba ocen na użytkownika:", avg_ratings_per_user)
print("Średnia liczba ocen na film:", avg_ratings_per_movie)
print("\nRozkład ocen:")
print(ratings_distribution)


Liczba wszystkich ocen: 100836
Liczba filmów w bazie: 9742
Liczba użytkowników: 610
Średnia liczba ocen na użytkownika: 165.30491803278687
Średnia liczba ocen na film: 10.350646684459043

Rozkład ocen:
rating
0.5     1370
1.0     2811
1.5     1791
2.0     7551
2.5     5550
3.0    20047
3.5    13136
4.0    26818
4.5     8551
5.0    13211
Name: count, dtype: int64


In [3]:
# Słownik tytułów filmów
movie_titles = dict(zip(movies['movieId'], movies['title']))

# Obliczenie C i m
C = ratings['rating'].mean()
m = ratings.groupby('movieId')['rating'].agg(['count', 'mean'])
m = m['count'].mean()

# Funkcja bayesian_avg
def bayesian_avg(ratings):
    bayesian_avg = (C * m + ratings.sum()) / (C + ratings.count())
    return bayesian_avg

# Obliczenie bayesian_avg_ratings
bayesian_avg_ratings = ratings.groupby('movieId')['rating'].agg(bayesian_avg).reset_index()
bayesian_avg_ratings.columns = ['movieId', 'bayesian_avg']

# Połączenie z danymi o filmach
movie_stats = movies.merge(bayesian_avg_ratings, on='movieId')

# Wyświetlenie wyników
print(movie_stats.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  bayesian_avg  
0  Adventure|Animation|Children|Comedy|Fantasy      4.024276  
1                   Adventure|Children|Fantasy      3.645857  
2                               Comedy|Romance      3.708193  
3                         Comedy|Drama|Romance      5.028823  
4                                       Comedy      3.558189  


In [4]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

# Wczytanie danych o ocenach i filmach
ratings = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv')
movies = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv')

# Liczba wszystkich ocen
total_ratings = ratings.shape[0]

# Liczba filmów w bazie
total_movies = movies.shape[0]

# Liczba użytkowników
total_users = ratings['userId'].nunique()

# Średnia liczba ocen na użytkownika
avg_ratings_per_user = total_ratings / total_users

# Średnia liczba ocen na film
avg_ratings_per_movie = total_ratings / total_movies

# Rozkład ocen
ratings_distribution = ratings['rating'].value_counts().sort_index()

# Wyświetlenie wyników
print("Liczba wszystkich ocen:", total_ratings)
print("Liczba filmów w bazie:", total_movies)
print("Liczba użytkowników:", total_users)
print("Średnia liczba ocen na użytkownika:", avg_ratings_per_user)
print("Średnia liczba ocen na film:", avg_ratings_per_movie)
print("\nRozkład ocen:")
print(ratings_distribution)

# Słownik tytułów filmów
movie_titles = dict(zip(movies['movieId'], movies['title']))

# Obliczenie C i m
C = ratings['rating'].mean()
m = ratings.groupby('movieId')['rating'].agg(['count', 'mean'])
m = m['count'].mean()

# Funkcja bayesian_avg
def bayesian_avg(ratings):
    bayesian_avg = (C * m + ratings.sum()) / (C + ratings.count())
    return bayesian_avg

# Obliczenie bayesian_avg_ratings
bayesian_avg_ratings = ratings.groupby('movieId')['rating'].agg(bayesian_avg).reset_index()
bayesian_avg_ratings.columns = ['movieId', 'bayesian_avg']

# Połączenie z danymi o filmach
movie_stats = movies.merge(bayesian_avg_ratings, on='movieId')

# Wyświetlenie wyników
print(movie_stats.head())

def create_sparse(df):
    M = df['userId'].nunique()
    N = df['movieId'].nunique()
    user_mapper = dict(zip(np.unique(df["userId"]), list(range(M))))
    movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(N))))
    user_inv_mapper = dict(zip(list(range(M)), np.unique(df["userId"])))
    movie_inv_mapper = dict(zip(list(range(N)), np.unique(df["movieId"])))
    user_index = [user_mapper[i] for i in df['userId']]
    item_index = [movie_mapper[i] for i in df['movieId']]
    X = csr_matrix((df["rating"], (user_index, item_index)), shape=(M,N))
    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

# Utworzenie macierzy rzadkiej
X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_sparse(ratings)

n_total = X.shape[0] * X.shape[1]
n_ratings = X.nnz
sparsity = n_ratings / n_total
sparsity_p = round(sparsity * 100, 2)

print(f"Matrix sparsity: {sparsity_p}%")


Liczba wszystkich ocen: 100836
Liczba filmów w bazie: 9742
Liczba użytkowników: 610
Średnia liczba ocen na użytkownika: 165.30491803278687
Średnia liczba ocen na film: 10.350646684459043

Rozkład ocen:
rating
0.5     1370
1.0     2811
1.5     1791
2.0     7551
2.5     5550
3.0    20047
3.5    13136
4.0    26818
4.5     8551
5.0    13211
Name: count, dtype: int64
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  bayesian_avg  
0  Adventure|Animation|Children|Comedy|Fantasy      4.024276  
1                   Adventure|Children|Fantasy      3.645857  
2                               Comedy|Romance      3.708193  
3                         Comedy|Drama|Romance      5.028823  
4                  

In [6]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

def find_similar_movies(movie_id, X, movie_mapper, movie_inv_mapper, k, metric='cosine'):
    X = X.T
    neighbour_ids = []
    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    if isinstance(movie_vec, (np.ndarray)):
        movie_vec = movie_vec.reshape(1, -1)
    kNN = NearestNeighbors(n_neighbors=k+1, algorithm="brute", metric=metric)
    kNN.fit(X)
    neighbour = kNN.kneighbors(movie_vec, return_distance=False)
    neighbour = neighbour[0][1:]  # Pomijamy pierwszy element, który jest samym filmem
    for n in neighbour:
        neighbour_ids.append(movie_inv_mapper[n])
    return neighbour_ids

# Przykładowe użycie funkcji do znalezienia 10 rekomendacji dla filmu o identyfikatorze 1 (Toy Story)
recommendations = find_similar_movies(1, X, movie_mapper, movie_inv_mapper, k=10)

# Wyświetlenie rekomendacji
for i, movie_id in enumerate(recommendations, 1):
    print(f"{i}. {movie_titles[movie_id]}")


1. Toy Story 2 (1999)
2. Jurassic Park (1993)
3. Independence Day (a.k.a. ID4) (1996)
4. Star Wars: Episode IV - A New Hope (1977)
5. Forrest Gump (1994)
6. Lion King, The (1994)
7. Star Wars: Episode VI - Return of the Jedi (1983)
8. Mission: Impossible (1996)
9. Groundhog Day (1993)
10. Back to the Future (1985)
